# Single-Tab Q&A with LangChain + Flan-T5

This notebook demonstrates a **free**, **local** approach to question-answering using:

- **LangChain** for chunking, embedding, and retrieval
- **Flan-T5** (a powerful open-source model from Google) for generating answers
- **Gradio** for an easy-to-use web interface

**Features**:
- Ingest one or more URLs
- Ask a question in the same interface
- Answers are grounded in the ingested content (with potential general knowledge from Flan-T5)

---

In [7]:
# Make sure you're using a GPU runtime:
# Runtime > Change runtime type > Hardware accelerator: GPU

!pip install langchain chromadb sentence-transformers transformers accelerate gradio faiss-cpu
!pip install -U langchain-community
!pip install -U bitsandbytes accelerate

In [4]:
import os
import gradio as gr
import requests
from bs4 import BeautifulSoup

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

############################
# 1) Load Flan-T5 model once (for faster usage)
############################

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True,
    torch_dtype=torch.float16  # use float16 to save memory
)

flan_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.0,  # reduce creativity for more factual answers
    do_sample=False
)

llm = HuggingFacePipeline(pipeline=flan_pipeline)

############################
# 2) Global references
############################
qa_chain = None
vectorstore = None

############################
# 3) Helper functions
############################

def scrape_text_from_url(url):
    """
    Scrape the URL for text (using <p> tags).
    Returns combined text or an error message.
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except Exception as e:
        return f"[Error fetching {url}]: {e}"

    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = [
        p.get_text(strip=True)
        for p in soup.find_all("p")
        if len(p.get_text(strip=True).split()) > 5
    ]
    text = "\n".join(paragraphs)
    return text

def build_vectorstore_from_urls(urls):
    """
    1) Scrape each URL.
    2) Split into chunks.
    3) Create a Chroma vectorstore.
    """
    combined_text = ""
    for url in urls.split(","):
        url = url.strip()
        if url:
            combined_text += scrape_text_from_url(url) + "\n"

    # Split text into chunks
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_text(combined_text)
    docs = [Document(page_content=c) for c in chunks if c.strip()]

    # Create embeddings + vector store
    embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vs = Chroma.from_documents(docs, embedding=embed)
    return vs

def ensure_chain_ready(urls):
    """
    If we haven't ingested or user provides new URLs,
    build a new vectorstore + QA chain.
    """
    global qa_chain, vectorstore
    if not urls.strip():
        return "Please enter at least one URL."

    # Build new vectorstore
    vectorstore = build_vectorstore_from_urls(urls)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    # Build QA chain
    new_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever
    )
    qa_chain = new_chain
    return "Content ingested."

############################
# 4) Main function for the single tab
############################
def ingest_and_ask(urls, question):
    """
    1) Ingest the URL(s) if needed (build chain).
    2) Ask question using the chain.
    """
    global qa_chain

    # Step 1: Build chain from URLs
    if not urls.strip():
        return "Please provide a URL.", ""

    ingest_message = ensure_chain_ready(urls)
    if not qa_chain:
        return "Could not build chain. Check URL(s).", ""

    # Step 2: Ask question
    if not question.strip():
        return ingest_message, "Please enter a question."

    answer = qa_chain.run(question)
    return ingest_message, answer

def clear_data():
    global qa_chain, vectorstore
    qa_chain = None
    vectorstore = None
    return "Ingested data cleared!"

############################
# 5) Build the Gradio UI (Single Tab)
############################
with gr.Blocks() as demo:
    gr.Markdown("# Web Q&A Tool\nIngest one or more URLs and ask a question in one go.")

    url_input = gr.Textbox(label="Enter URLs (comma-separated)")
    question_input = gr.Textbox(label="Ask a question")
    submit_button = gr.Button("Submit")
    ingest_status = gr.Textbox(label="Ingestion Status", interactive=False)
    answer_output = gr.Textbox(label="Answer", interactive=False)

    submit_button.click(
        fn=ingest_and_ask,
        inputs=[url_input, question_input],
        outputs=[ingest_status, answer_output]
    )

    with gr.Row():
        clear_button = gr.Button("Clear Data")
        clear_status = gr.Textbox(label="Status", interactive=False)
        clear_button.click(clear_data, inputs=[], outputs=clear_status)

demo.launch()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a946d18ab1408047fb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Usage Instructions

1. **Enter URLs**: Provide one or more comma-separated URLs (e.g. `https://www.example.com, https://www.second.com`).
2. **Type a question** about the content from those URLs.
3. **Submit**: The app scrapes + ingests the URLs (if not already), then returns an answer.
4. **Clear Data**: Resets the system so you can ingest new content.

**Tip**: If ingestion is slow, try a smaller model (e.g., `google/flan-t5-small`), or reduce chunk size.
